# 管理者の追加

---

CoursewareHubの管理ユーザをローカルユーザとして追加します。

## はじめに

このNotebookは、CoursewareHub環境構築の際に誤った管理者情報を登録したなどの理由により、CoursewareHub環境に管理者としてログインできるユーザが存在しないなどの状況を回避するためのものです。管理者としてログインできるユーザが存在する場合は **JupyterHub のAdmin画面から管理者の登録を行ってください**。

## UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

操作対象となるUnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

追加する管理者の情報を指定します。

管理者のメールアドレスを指定してください。

In [ ]:
# (例)
# teacher_email = 'admin@example.org'

teacher_email = 

管理者のパスワードを指定してください。

> ここで指定したパスワードは CoursewareHub にログインする際に必要となります。

In [ ]:
from getpass import getpass
teacher_password = getpass()

## 管理者の追加

### Systemユーザの作成

CoursewareHubではメールアドレスから一定のルールで導き出された名前を、ローカルユーザ名として利用します。管理ユーザのローカルユーザ名を確認します。

In [ ]:
import hashlib
import re

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except alphabetic
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h
    return result

In [ ]:
teacher_id = get_username_from_mail_address(teacher_email)
print(teacher_id)

ホームディレクトリの親ディレクトリを作成します。

In [ ]:
!ansible {target_hub} -b -m file -a 'path=/jupyter/users state=directory'

管理者のホームディレクトリを変数に設定しておきます。

In [ ]:
teacher_homedir = f'/jupyter/users/{teacher_id}'
print(teacher_homedir)

`manager`ノードでユーザを作成します。

In [ ]:
!ansible {target_hub} -b -m user -a 'name={teacher_id} home={teacher_homedir}'

ホームディレクトリが作成されていることを確認します。

In [ ]:
!ansible {target_hub} -b -a 'ls -la {teacher_homedir}'

`worker`ノードにも同じ名前のユーザを作成します。ホームディレクトリはNFSになるので、`manager` のUID/GIDと同じ値でユーザを作成します。

まず、`manager` での UID/GID の値を確認します。

In [ ]:
lines = !ansible -a 'id -u {teacher_id}' {target_hub}
teacher_uid = lines[1]
lines = !ansible -a 'id -g {teacher_id}' {target_hub}
teacher_gid = lines[1]
lines = !ansible -a 'id -g -n {teacher_id}' {target_hub}
teacher_group = lines[1]
(teacher_uid, teacher_gid, teacher_group)

`worker`ノードでグループを作成します。

> GID を指定するので、まずグループを作成します。

In [ ]:
!ansible {target_nodes} -b -m group -a 'name={teacher_group} gid={teacher_gid}'

`worker`ノードでユーザを作成します。

In [ ]:
!ansible {target_nodes} -b -m user \
    -a 'name={teacher_id} uid={teacher_uid} group={teacher_group} \
        home={teacher_homedir}'

ユーザが作成されたことを確認します。

In [ ]:
!ansible {target_hub} -a 'id {teacher_id}'
!ansible {target_nodes} -a 'id {teacher_id}'

### Prepare contents directory

コンテンツを格納するディレクトリ `info`, `textbook` を準備します。

In [ ]:
for x in ['info', 'textbook']:
    !ansible {target_hub} -b -m file \
        -a 'path={teacher_homedir}/{x} state=directory \
            owner={teacher_uid} group={teacher_group} mode=0777'

### Create SSH key and register

JupyterHubを構成するマシンへログインするための鍵の生成と登録を行います。

SSHの鍵ペアを作成します。

In [ ]:
!ansible {target_hub} -b -a 'creates={teacher_homedir}/.ssh/id_rsa \
    sudo -u {teacher_id} \
    ssh-keygen -N "" -f {teacher_homedir}/.ssh/id_rsa'

鍵ファイルが作成されたことを確認します。

In [ ]:
!ansible {target_hub} -b -m shell -a 'ls -l {teacher_homedir}/.ssh/id_rsa*'

作成した公開鍵を `authorized_keys` に登録します。まず、公開鍵の値を取得します。

In [ ]:
lines = !ansible {target_hub} -b -a 'cat {teacher_homedir}/.ssh/id_rsa.pub'
pubkey = lines[1]
print(pubkey)

`authorized_keys`に登録します。

In [ ]:
!ansible {target_hub} -b -m authorized_key -a 'user={teacher_id} key="{pubkey}"'

ユーザーのホームディレクトリに不適切なpermissionが設定されているとsshの鍵認証に失敗するので、妥当な値が設定されていることを保証しておきます。

In [ ]:
!ansible {target_hub} -b -m file \
    -a 'path={teacher_homedir} mode="0755" \
        owner={teacher_id} group={teacher_group}'

### Grant sudo

JupyterHubを構成するマシン上でのsudo権限を与える設定ファイルを配置します。

事前のチェックを行います。

In [ ]:
!ansible {target_hub} -CD -b -m lineinfile \
    -a 'dest=/etc/sudoers.d/{teacher_id} create=yes\
        line="{teacher_id} ALL=(ALL) NOPASSWD: ALL"'

実際に設定ファイルの配置を行います。

In [ ]:
!ansible {target_hub} -b -m lineinfile \
    -a 'dest=/etc/sudoers.d/{teacher_id} create=yes\
        line="{teacher_id} ALL=(ALL) NOPASSWD: ALL"'

### Set ansible inventory

JupyterHubを構成するマシンを操作するためのインベントリを配布します。

まずは、インベントリ配布するための playbook をチェックモード実行します。

In [ ]:
!ansible-playbook -CDv -l {target_hub} \
    -e teacher_id={teacher_id} -e teacher_homedir={teacher_homedir} \
    -e target_hub={target_hub} -e target_nodes={target_nodes} \
    playbooks/deploy-inventory.yml \
    || true

実際にインベントリを構築環境の`~/ansible/inventory`に配布します。

In [ ]:
!ansible-playbook -Dv -l {target_hub} \
    -e teacher_id={teacher_id} -e teacher_homedir={teacher_homedir} \
    -e target_hub={target_hub} -e target_nodes={target_nodes} \
    playbooks/deploy-inventory.yml

### JupyterHubユーザの作成


直接DBを変更してユーザーを登録します。

まず、JupyterHubのユーザテーブルにユーザを追加するための SQL ファイルを作成します。

In [ ]:
import random
from datetime import datetime
from tempfile import TemporaryDirectory

cookie_id = ''.join(random.choices("0123456789abcdef", k=32))
with TemporaryDirectory() as work_dir:
    sql_file = Path(work_dir) / ('create_user_{:%Y%m%d_%H%M%S}.sql'.format(datetime.now()))
    with sql_file.open(mode='w') as f:
        f.write(f'''
INSERT INTO users (name, admin, cookie_id, last_activity)
  VALUES ('{teacher_id}', TRUE, '{cookie_id}', '{datetime.now().isoformat()}');
''')
    !cat {str(sql_file)}
    !ansible {target_hub} -b -m copy -a 'src={str(sql_file)} dest=/jupyter/psql/init/'

PostgreSQLコンテナのコンテナIDと実行しているホストのIPアドレスを取得します。

In [ ]:
lines = !ansible {target_hub} -b -a 'docker service ps {{{{ugroup_name}}}}_postgres -q'
sid = lines[1]

lines = !ansible {target_hub} -b -a \
    'docker inspect --format "{{% raw %}} {{{{.NodeID}}}} {{{{.Status.ContainerStatus.ContainerID}}}} {{% endraw %}}" {sid}'
nodeid, cid = lines[1].split()
print(cid)

lines = !ansible {target_hub} -b -a \
    'docker node inspect --format "{{% raw %}}{{{{.Status.Addr}}}} {{% endraw %}}" {nodeid}'
target_ip = lines[1].split()[0]
print(target_ip)

SQLファイルを実行します。

In [ ]:
!ansible {target_ip} -b -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -f /docker-entrypoint-initdb.d/{sql_file.name}'

ユーザが登録されたことを確認します。

In [ ]:
!ansible {target_ip} -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -c "SELECT * FROM users"'

ローカルユーザーのテーブルにも登録します。 SQLファイルを作成します。

In [ ]:
from crypt import crypt

with TemporaryDirectory() as work_dir:
    sql2_file = Path(work_dir) / ('create_localuser_{:%Y%m%d_%H%M%S}.sql'.format(datetime.now()))
    with sql2_file.open(mode='w') as f:
        f.write(f'''
INSERT INTO local_users VALUES(
  nextval('local_users_id_seq'),
  '{teacher_id}',
  '{crypt(teacher_password)}',
  '{teacher_email}'
);
''')
    !ansible {target_hub} -b -m template -a 'src={str(sql2_file)} dest=/jupyter/psql/init/'
    !ansible {target_hub} -b -a 'cat /jupyter/psql/init/{str(sql2_file.name)}'

SQLファイルを実行します。

In [ ]:
!ansible {target_ip} -b -a 'docker exec -i {cid} \
    psql -d {{{{db_name}}}} -U {{{{db_user}}}} -f /docker-entrypoint-initdb.d/{sql2_file.name}'

## コンテンツの配備の準備

CoursewareHubのコンテンツを格納するディレクトリを作成し、コンテンツの配置を行うNotebookを管理者のホームディレクトリに配置します。

実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -e teacher_id={teacher_id} -CDv playbooks/manage-tools.yml

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} -e teacher_id={teacher_id} playbooks/manage-tools.yml

## CoursewareHubにアクセスする

追加した管理者ユーザでログインできることを確認してください。

次のセルを実行するとCoursewareHubのアドレスを表示します。

In [ ]:
out = !ansible {target_ip} -c local -a 'echo https://{{{{master_fqdn}}}}'
print(out[1])

CoursewareHubのコンテンツを配備するためのNotebook「CoursewareHubコンテンツの配備.ipynb」がログイン環境に配置されています。CoursewareHub環境構築後の最初の管理者としてログインした場合は実行してください。既に他の管理者によって実行されている場合は不要です。